Import necessary libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
weather = pd.read_csv("../data/weather.csv")

In [8]:
weather.head()

,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,...,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_conditions,hour_icon,hour_source,hour_stations,datetime,sunrise_datetime,sunset_datetime
0,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Overcast,snow,obs,remote,2022-02-24 00:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
1,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Partially cloudy,fog,obs,remote,2022-02-24 01:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
2,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,NaN,NaN,NaN,Overcast,cloudy,obs,33177099999,2022-02-24 02:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
3,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Overcast,fog,obs,remote,2022-02-24 03:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
4,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Overcast,cloudy,obs,remote,2022-02-24 04:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06


# Analyze Data

## Weather Dataset

### Weather Dataset Overview


In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608304 entries, 0 to 608303
Data columns (total 65 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   city_latitude         608304 non-null  float64
 1   city_longitude        608304 non-null  float64
 2   city_resolvedAddress  608304 non-null  object 
 3   city_address          608304 non-null  object 
 4   city_timezone         608304 non-null  object 
 5   city_tzoffset         608304 non-null  float64
 6   day_datetime          608304 non-null  object 
 7   day_datetimeEpoch     608304 non-null  int64  
 8   day_tempmax           608304 non-null  float64
 9   day_tempmin           608304 non-null  float64
 10  day_temp              608304 non-null  float64
 11  day_feelslikemax      608304 non-null  float64
 12  day_feelslikemin      608304 non-null  float64
 13  day_feelslike         608304 non-null  float64
 14  day_dew               608304 non-null  float64
 15  


The output of `weather.info()` reveals that this dataset contains a **significant number of missing values** across multiple columns.

Because of this, each column should be **individually evaluated** to determine the appropriate handling strategy:
- **Impute missing values**
- **Keep as-is**
- Or **drop the column entirely**

In addition, both the `day_datetime` and `hour_datetime` columns are currently stored as `object` types.
For proper time-based analysis, we will:
- Convert them to `datetime` format,
- And combine them into a single unified `datetime` column to simplify processing and ensure consistency.

There are also time-related columns such as `day_sunrise` and `day_sunset`, which will also be converted to proper `datetime` format.

In [5]:
weather['datetime'] = pd.to_datetime(
    weather['day_datetime'].astype(str) + ' ' + weather['hour_datetime'].astype(str),
    format='%Y-%m-%d %H:%M:%S'
)

weather['sunrise_datetime'] = pd.to_datetime(
    weather['day_datetime'].astype(str) + ' ' + weather['day_sunrise'].astype(str),
    format='%Y-%m-%d %H:%M:%S'
)

weather['sunset_datetime'] = pd.to_datetime(
    weather['day_datetime'].astype(str) + ' ' + weather['day_sunset'].astype(str),
    format='%Y-%m-%d %H:%M:%S'
)
weather.drop(['day_datetime', 'hour_datetime', "day_sunset", "day_sunrise"], axis=1, inplace=True)

### Understanding the Method for Handling Missing Values

Before applying any filling or deletion strategy, it's important to **fully understand the context and structure** of missing data in the dataset.

Our approach consists of the following steps:

1. **Explore each feature**:
   Determine the percentage and pattern of missing values in each column.
   Some columns may have isolated gaps (suitable for filling), while others may be mostly empty (better to drop).

2. **Evaluate relevance**:
   Assess whether the column is important for further analysis or modeling.
   Columns with low importance or no correlation may be dropped regardless of missingness.

3. **Check the duration and nature of gaps**:
   For time-series data, long gaps might require different handling than short, isolated ones.
   Short gaps are safely fillable with **`ffill`/`bfill`**.

4. **Apply missing value treatment after reviewing all features**:
   This ensures that the method is applied **consistently across the dataset**, avoiding unnecessary fragmentation or bias.

Only after this full review, we will proceed with filling or removing missing values based on the nature of each column.


In [6]:
def find_longest_missing_gap(df, target_column: str):
    """
    Finds and prints the longest continuous time gap where the values are missing (NaN)
    in a specified column of a DataFrame.

    Parameters:
    - df (pd.DataFrame): The dataset containing the time and target columns.
    - target_column (str): The name of the column to check for missing values.

    Returns:
    - None. Prints the start, end, and duration of the longest missing interval.
    """

    df = df.copy()
    df = df.sort_values("datetime")
    missing = df[target_column].isna()
    group_id = missing.ne(missing.shift()).cumsum()

    missing_groups = df[missing].groupby(group_id)

    gap_info = missing_groups["datetime"].agg(['first', 'last'])
    gap_info['duration'] = gap_info['last'] - gap_info['first']

    if not gap_info.empty:
        longest = gap_info.loc[gap_info['duration'].idxmax()]
        print("Longest period with missing values in column:", target_column)
        print(f"– From: {longest['first']}")
        print(f"– To  : {longest['last']}")
    else:
        print(f"No missing values found in column '{target_column}'.")

#### Checking for Missing Values

1) **Visibility**


In [7]:
find_longest_missing_gap(weather, "day_visibility")

Longest period with missing values in column: day_visibility
– From: 2024-03-31 02:00:00
– To  : 2024-03-31 04:00:00


We can see that the longest gap in this column is just **2 hours**,
which makes it reasonable to apply either **forward fill (`ffill`)** or **backward fill (`bfill`)** as an effective way to handle the missing values without introducing significant distortion.

Additionally, we have a corresponding feature, `hour_visibility`, so let's examine that as well.

In [9]:
find_longest_missing_gap(weather, "hour_visibility")

Longest period with missing values in column: hour_visibility
– From: 2023-03-26 02:00:00
– To  : 2023-03-26 04:00:00


We can see that the longest gap in both columns is just **2 hours**. Let’s now check how they interact with each other.

In [11]:
weather[['datetime', "day_visibility", "hour_visibility"]]

,datetime,day_visibility,hour_visibility
0,2022-02-24 00:00:00,12.2,0.0
1,2022-02-24 01:00:00,12.2,0.2
2,2022-02-24 02:00:00,12.2,10.0
3,2022-02-24 03:00:00,12.2,0.1
4,2022-02-24 04:00:00,12.2,0.0
...,...,...,...
608299,2025-03-01 19:00:00,7.0,NaN
608300,2025-03-01 20:00:00,7.0,10.0
608301,2025-03-01 21:00:00,7.0,NaN
608302,2025-03-01 22:00:00,7.0,NaN


We can notice that `day_visibility` has the same value throughout the entire day. This might suggest that it represents a fixed value, possibly the mean of `hour_visibility`. Let’s check it now.

In [12]:
weather["hour_visibility"].iloc[:24].mean()

np.float64(12.154166666666667)

So, we were correct in assuming that `day_visibility` represents the mean of `hour_visibility`. Since we will be storing the data on an hourly basis, the `day_visibility` column is unnecessary for our model. Therefore, we will drop this column.

Additionally, we noticed that the `hour_visibility` column has missing values, which we will address by applying the **forward fill (`ffill`)** method.

2) **Solarradiation, Solarenergy, UVindex**


In [13]:
find_longest_missing_gap(weather, "day_solarradiation")

Longest period with missing values in column: day_solarradiation
– From: 2024-10-28 00:00:00
– To  : 2024-10-28 23:00:00


In [14]:
find_longest_missing_gap(weather, "day_solarenergy")

Longest period with missing values in column: day_solarenergy
– From: 2024-10-28 00:00:00
– To  : 2024-10-28 23:00:00


In [15]:
find_longest_missing_gap(weather, "day_uvindex")

Longest period with missing values in column: day_uvindex
– From: 2024-10-28 00:00:00
– To  : 2024-10-28 23:00:00


These three columns share the same missing value pattern, including identical gap locations and durations.
The longest missing stretch is around **1 day**, which is not a small gap — but also not too long to make reconstruction unreliable.

Simple methods like `ffill` or `bfill` could work.

Let's now examine corresponding features: `hour_solarradiation`,`hour_solarenergy` and `hour_uvindex`.

In [17]:
find_longest_missing_gap(weather, "hour_solarradiation")

Longest period with missing values in column: hour_solarradiation
– From: 2024-10-28 00:00:00
– To  : 2024-10-28 23:00:00


In [16]:
find_longest_missing_gap(weather, "hour_solarenergy")

Longest period with missing values in column: hour_solarenergy
– From: 2024-10-28 00:00:00
– To  : 2024-10-28 23:00:00


In [18]:
find_longest_missing_gap(weather, "hour_uvindex")

Longest period with missing values in column: hour_uvindex
– From: 2024-10-28 00:00:00
– To  : 2024-10-28 23:00:00


We can see that these features have the same largest gap. Let's check how they interact with each other.


In [19]:
weather[['datetime', "day_solarradiation", "hour_solarradiation", "day_solarenergy", "hour_solarenergy", "day_uvindex",
         "hour_uvindex"]]

,datetime,day_solarradiation,hour_solarradiation,day_solarenergy,hour_solarenergy,day_uvindex,hour_uvindex
0,2022-02-24 00:00:00,36.9,0.0,2.8,NaN,1.0,0.0
1,2022-02-24 01:00:00,36.9,0.0,2.8,NaN,1.0,0.0
2,2022-02-24 02:00:00,36.9,NaN,2.8,NaN,1.0,NaN
3,2022-02-24 03:00:00,36.9,0.0,2.8,NaN,1.0,0.0
4,2022-02-24 04:00:00,36.9,0.0,2.8,NaN,1.0,0.0
...,...,...,...,...,...,...,...
608299,2025-03-01 19:00:00,106.1,0.0,9.2,0.0,4.0,0.0
608300,2025-03-01 20:00:00,106.1,0.0,9.2,0.0,4.0,0.0
608301,2025-03-01 21:00:00,106.1,0.0,9.2,0.0,4.0,0.0
608302,2025-03-01 22:00:00,106.1,0.0,9.2,0.0,4.0,0.0


In [20]:
weather["hour_solarradiation"].iloc[:24].mean()

np.float64(36.9)

In [22]:
weather["hour_solarenergy"].iloc[:24].sum()

np.float64(2.8)

In [24]:
weather["hour_uvindex"].iloc[:24].max()

np.float64(1.0)

From the basic analysis, we can see that `day_solarradiation` is the mean of `hour_solarradiation` for the given day, `day_solarenergy` is the sum of `hour_solarenergy`, and `day_uvindex` is the maximum of `hour_uvindex`. Following the same logic as with the previous features, the `day_` attributes are redundant and unnecessary for our model.

3) **Preciptype**


In [25]:
find_longest_missing_gap(weather, "day_preciptype")

Longest period with missing values in column: day_preciptype
– From: 2024-04-29 23:00:00
– To  : 2024-05-05 00:00:00


In [26]:
weather["day_preciptype"].value_counts()

day_preciptype
rain                      244492
rain;snow                  77389
snow                       18790
rain;freezingrain;snow      1104
freezingrain;snow            168
rain;freezingrain             72
ice                           24
freezingrain;snow;ice         24
rain;snow;ice                 24
Name: count, dtype: int64

The `day_preciptype` column contains meaningful categorical information about types of precipitation, including multiple combined values (e.g., "rain;snow").

Although ~44% of the values are missing, it's reasonable to assume they indicate **no precipitation**.

Therefore, instead of dropping or ignoring the column, we will keep it and **fill missing values with `"none"`** to preserve clarity and make the data ready for modeling or analysis.


In [27]:
weather["hour_preciptype"].value_counts()

hour_preciptype
['rain']            43309
['rain', 'snow']    12162
['snow']             9268
['freezingrain']      122
['ice']                 4
Name: count, dtype: int64

In [28]:
weather[['datetime', "day_preciptype", "hour_preciptype"]]

,datetime,day_preciptype,hour_preciptype
0,2022-02-24 00:00:00,snow,['snow']
1,2022-02-24 01:00:00,snow,['snow']
2,2022-02-24 02:00:00,snow,['snow']
3,2022-02-24 03:00:00,snow,['snow']
4,2022-02-24 04:00:00,snow,['snow']
...,...,...,...
608299,2025-03-01 19:00:00,NaN,NaN
608300,2025-03-01 20:00:00,NaN,NaN
608301,2025-03-01 21:00:00,NaN,NaN
608302,2025-03-01 22:00:00,NaN,NaN


As with the previous features, the `day_` attributes are linked to their corresponding hourly values, so we will drop the `day_preciptype` column. Additionally, we observe that `hour_preciptype` stores data in a list format, so it’s better to convert it to a string and process it later.

In [29]:
find_longest_missing_gap(weather, "hour_precip")

Longest period with missing values in column: hour_precip
– From: 2022-03-01 02:00:00
– To  : 2022-03-01 02:00:00


In [30]:
weather[weather["hour_precip"] == 0].head()

,city_latitude,city_longitude,city_resolvedAddress,city_address,city_timezone,city_tzoffset,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,...,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_conditions,hour_icon,hour_source,hour_stations,datetime,sunrise_datetime,sunset_datetime
0,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Overcast,snow,obs,remote,2022-02-24 00:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
1,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Partially cloudy,fog,obs,remote,2022-02-24 01:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
2,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,NaN,NaN,NaN,Overcast,cloudy,obs,33177099999,2022-02-24 02:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
3,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Overcast,fog,obs,remote,2022-02-24 03:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06
4,50.7469,25.3263,"Луцьк, Луцький район, Україна","Lutsk,Ukraine",Europe/Kiev,2.0,1645653600,4.9,0.7,2.6,...,0.0,NaN,0.0,Overcast,cloudy,obs,remote,2022-02-24 04:00:00,2022-02-24 07:13:36,2022-02-24 17:51:06


Since some values in this column are already equal to `0`, and the total amount of missing data is small,
with the longest missing gap being just **1 hour**, we will use **forward fill (`ffill`)** as a suitable method to handle the missing values.

Let's also compare it with `day_precip`.

In [31]:
weather[["datetime", "hour_precip", "day_precip"]]

,datetime,hour_precip,day_precip
0,2022-02-24 00:00:00,0.0,0.118
1,2022-02-24 01:00:00,0.0,0.118
2,2022-02-24 02:00:00,0.0,0.118
3,2022-02-24 03:00:00,0.0,0.118
4,2022-02-24 04:00:00,0.0,0.118
...,...,...,...
608299,2025-03-01 19:00:00,0.0,0.000
608300,2025-03-01 20:00:00,0.0,0.000
608301,2025-03-01 21:00:00,0.0,0.000
608302,2025-03-01 22:00:00,0.0,0.000


We can see that `day_precip` is simply the sum of `hour_precip`. Following the same logic as with the previous features, we will drop the `day_precip` column.

### Filling Missing Data

In [32]:
# 1. Categorical columns: fill with 'none'
weather["hour_preciptype"] = weather["hour_preciptype"].fillna("none")

# 2. Columns to forward fill
ffill_columns = ["hour_precip", "hour_visibility",
                 "hour_solarradiation", "hour_solarenergy", "hour_uvindex"
                 ]
weather[ffill_columns] = weather.groupby("city_address")[ffill_columns].ffill()

In [33]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608304 entries, 0 to 608303
Data columns (total 64 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   city_latitude         608304 non-null  float64       
 1   city_longitude        608304 non-null  float64       
 2   city_resolvedAddress  608304 non-null  object        
 3   city_address          608304 non-null  object        
 4   city_timezone         608304 non-null  object        
 5   city_tzoffset         608304 non-null  float64       
 6   day_datetimeEpoch     608304 non-null  int64         
 7   day_tempmax           608304 non-null  float64       
 8   day_tempmin           608304 non-null  float64       
 9   day_temp              608304 non-null  float64       
 10  day_feelslikemax      608304 non-null  float64       
 11  day_feelslikemin      608304 non-null  float64       
 12  day_feelslike         608304 non-null  float64       
 13 

Currently, only one column still contains missing values and won't be dropped,
so we will apply the **backfill (`bfill`)** method specifically to that column to handle the remaining gaps.

In [34]:
weather["hour_solarenergy"] = weather.groupby("city_address")["hour_solarenergy"].bfill()

In [35]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608304 entries, 0 to 608303
Data columns (total 64 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   city_latitude         608304 non-null  float64       
 1   city_longitude        608304 non-null  float64       
 2   city_resolvedAddress  608304 non-null  object        
 3   city_address          608304 non-null  object        
 4   city_timezone         608304 non-null  object        
 5   city_tzoffset         608304 non-null  float64       
 6   day_datetimeEpoch     608304 non-null  int64         
 7   day_tempmax           608304 non-null  float64       
 8   day_tempmin           608304 non-null  float64       
 9   day_temp              608304 non-null  float64       
 10  day_feelslikemax      608304 non-null  float64       
 11  day_feelslikemin      608304 non-null  float64       
 12  day_feelslike         608304 non-null  float64       
 13 

With this, our missing value handling is complete.
While it's possible that some information was lost during the process, this is often an acceptable trade-off for achieving a clean and consistent dataset suitable for further analysis and modeling.

## Feature Analysis

### Analysis of Day_ Features

As we observed earlier, the `day_` features are closely related to the `hour_` features. Since we store data on an hourly basis, these `day_` features are not necessary for our analysis. Let's now examine the other `day_` features to determine if they are required for our future analysis.


1) **Temperature** (`tempmax`, `tempmin`, `temp`, `feelslike`, `feelslikemax`, `feelslikemin`)


These features are self-explanatory based on their names. Temperature-related attributes, such as `tempmax`, `tempmin`, `temp`, are valuable for our analysis. However, the `feelslike` attributes (including `hour_feelslike`) are less relevant, as they represent subjective perceptions of temperature, whereas we are interested in objective data (factual values).

2) **Dew, Humidity**

In [36]:
weather[["datetime", "day_dew", "hour_dew", "day_humidity", "hour_humidity"]]

,datetime,day_dew,hour_dew,day_humidity,hour_humidity
0,2022-02-24 00:00:00,0.0,0.8,83.7,89.18
1,2022-02-24 01:00:00,0.0,0.6,83.7,87.90
2,2022-02-24 02:00:00,0.0,1.2,83.7,88.58
3,2022-02-24 03:00:00,0.0,0.3,83.7,86.63
4,2022-02-24 04:00:00,0.0,0.1,83.7,87.85
...,...,...,...,...,...
608299,2025-03-01 19:00:00,-5.9,-5.5,81.2,73.09
608300,2025-03-01 20:00:00,-5.9,-4.6,81.2,81.17
608301,2025-03-01 21:00:00,-5.9,-6.1,81.2,68.31
608302,2025-03-01 22:00:00,-5.9,-6.2,81.2,71.36


In [39]:
weather["hour_dew"].iloc[:24].mean()

np.float64(0.04583333333333339)

In [40]:
weather["hour_humidity"].iloc[:24].mean()

np.float64(83.66458333333333)

Since `day_dew` and `day_humidity` simply represent the mean of the corresponding hourly values, they can be easily dropped.

In [ ]:
weather.drop(["day_visibility", "day_solarenergy", "day_solarradiation", "day_uvindex", "day_preciptype", "day_precip",
              "hour_feelslike", "day_feelslikemax", "day_feelslikemin", "day_feelslike", "day_dew", "day_humidity"],
             axis=1,
             inplace=True)